# Aula 1 - Validação cruzada e aleatoriedade inicial

In [2]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

Cross-Validation - Espécie de validação que replica a separação dos dados de treino e de teste diversas vezes afim de minimizar o efeito da aleatóriedade nos modelos. 

K-fold - separa um número K de dados para teste e testa K vezes o modelo, podendo assim tirar a média de acurácia dos testes

Leave One Out - separa um único valor dos dados para teste e testa o modelo para cada valor separadamente.

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

SEED = 5
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y) # Stratify = Divide os dados de x e y de acordo com a proporção dos dados y, que foi passado como parametro

print(f'Será treinado com {len(treino_y)} elementos e testado com {len(teste_y)}')

Será treinado com 7500 elementos e testado com 2500


In [22]:
from sklearn.dummy import DummyClassifier

SEED = 158020
np.random.seed(SEED)

dummy_stratifier = DummyClassifier( )
dummy_stratifier.fit(treino_x, treino_y)
acuracia_dummy = dummy_stratifier.score(teste_x, teste_y) * 100

print('A acurácia foi de %.2f%%' % acuracia_dummy)

A acurácia foi de 50.96%


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

SEED = 355
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('Acuracia foi de %.2f%%' % acuracia)

Acuracia foi de 76.84%


Fazendo modelo com Cross_Validation

In [50]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print('Acurácia média %.2f' % (media * 100))
    print('O intervalo de confiança deste modelo é entre %.2f, %.2f' % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

In [51]:
from sklearn.model_selection import cross_validate

SEED = 5
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78
O intervalo de confiança deste modelo é entre 75.21, 76.35


#Aula 2 - KFold e aleatoriedade

Utilizando o módulo KFold é possivel implantar a aleatóriedade na separação dos dados, ao invés de separa-los por grupos fixos

In [52]:
from sklearn.model_selection import KFold

SEED = 158135
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78
O intervalo de confiança deste modelo é entre 73.50, 78.06


# Aula 03 - Estratificação

Caso seja necessario separar os dados de acordo com a proporção da distribuição deles no dataset (como foi usado como parâmetro no train_teste_split deste notebook), podemos usar a biblioteca StratifiedKFold, que seleciona grupos de dados considerando suas proporções no dataset.

In [53]:
from sklearn.model_selection import StratifiedKFold

SEED = 158135
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78
O intervalo de confiança deste modelo é entre 73.87, 77.69


IMPORTANTE - A ideia é que a validação cruzada num conjunto de dados somente diz o que você pode esperar. Ela não treina o seu modelo final.
Para treinar realmente é preciso fazer um fit do modelo:


```
modelo.fit(x, y)
```



# Aula 4 - Dados Agrupáveis

É possivel também fazer o cross validation considerando que os dados podem ter agrupamentos diferentes. No exemplo abaixo, é acrescentado uma coluna 'modelo', simulando um agrupamento dos dados por modelo do carro

In [57]:
np.random.seed(SEED)
np.random.randint(-2,2)

dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,2, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,21
1,40557.96,1,20,12622.05362,23
2,89627.50,0,12,11440.79806,13
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,5


In [59]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média 75.78
O intervalo de confiança deste modelo é entre 70.97, 80.60


# Aula 5 - Pipeline de treino e validação

Por fim, quando precisamos re-escalar os dados para alguns modelos como o SVC (que é sensível a escala diferente dos dados), fazemos o seguinte:

In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaller = StandardScaler()
scaller.fit(treino_x)
treino_x_scaller = scaller.transform(treino_x)
teste_x_scaller = scaller.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_scaller, treino_y)
previsoes = modelo.predict(teste_x_scaller)

acuracia = accuracy_score(teste_y, previsoes) * 100
print('Acuracia foi de %.2f%%' % acuracia)

Acuracia foi de 77.48%


Porém, para usar o cross validate, não podemos escalar os dados só uma vez, pois escalamos apenas com os dados de treino, logo, como os dados de treino estão sendo alterados a todo momento no cross validate, temos que alterar o StandartScaller a cada novo cross validate.

Para isso usamos o método pipeline que, a cada vez que é feita a separação pelo cross validate, é feito também o standartscaler e rodado o modelo sobre os novos dados separados

In [63]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaller = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaller), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(pipeline, x, y, cv=cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média 76.72
O intervalo de confiança deste modelo é entre 71.98, 81.46
